## Lesson 3: A Practical Guide to Arcpy

### Exercise 1: Steps to Automate Any Task
In this exercise you will begin taking steps to automate a common workflow using Arcpy. We'll walk through a common process that many image analysts perform: loading imagery into ArcGIS Pro, creating a mosaic dataset, then clipping the mosaic dataset to a Region of Interest.

#### Create Mosaic Dataset

1. Open the 'Create Mosaic Dataset' Geoprocessing Tool.
2. Input the following parameters:
    - **Output Location** - arcpy_workshop.gdb
    - **Mosaic Dataset Name** - Kangaroo_Island_2019
    - **Coordinate System** - WGS_1984_UTM_Zone_53S
    - **Product Definition** - None
3. WIthout running the tool, select the 'Copy Python Command' button and paste the results below
*Note: Your file paths will be different than the solution*

In [1]:
# Enter Solution

<details>
<summary> <b> Reveal Answer </b> </summary>

    arcpy.management.CreateMosaicDataset(
        in_workspace=r"C:\Users\col12422\Documents\ArcGIS\Projects\Arcpy Workshop\Arcpy Workshop.gdb",
        in_mosaicdataset_name="Kangaroo_Island_2019",
        coordinate_system='PROJCS["WGS_1984_UTM_Zone_53S",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",10000000.0],PARAMETER["Central_Meridian",135.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
        num_bands=None,
        pixel_type="",
        product_definition="NONE",
        product_band_definitions=None
    )
</details>

#### Add Rasters to Mosaic Dataset

1. Open the 'Add Rasters to Mosaic Dataset' Geoprocessing Tool.
2. Input the following parameters:
    - **Mosaic Dataset** - Kangaroo_Island_2019
    - **Raster Type** - Raster Dataset
    - **Processing Templates** - Default
    - **Input Data** - File
        - *Browse* -> Select Kangaroo_Island_2019 A & B (Located in Arcpy Workshop.gdb)
    - **Raster Processing** - Check the boxes for 'Calculate Statistics' & 'Build Raster Pyramids'
3. WIthout running the tool, select the 'Copy Python Command' button and paste the results below
*Note: Your file paths will be different than the solution*

In [16]:
# Enter Solution

<details>
<summary> <b> Reveal Answer </b> </summary>

    arcpy.management.AddRastersToMosaicDataset(
        in_mosaic_dataset="Kangaroo_Island_2019",
        raster_type="Raster Dataset",
        input_path=r"'C:\Users\col12422\Documents\ArcGIS\Projects\Arcpy Workshop\Arcpy Workshop.gdb\Kangaroo_Island_2019A';'C:\Users\col12422\Documents\ArcGIS\Projects\Arcpy Workshop\Arcpy Workshop.gdb\Kangaroo_Island_2019B'",
        update_cellsize_ranges="UPDATE_CELL_SIZES",
        update_boundary="UPDATE_BOUNDARY",
        update_overviews="NO_OVERVIEWS",
        maximum_pyramid_levels=None,
        maximum_cell_size=0,
        minimum_dimension=1500,
        spatial_reference=None,
        filter="",
        sub_folder="SUBFOLDERS",
        duplicate_items_action="ALLOW_DUPLICATES",
        build_pyramids="BUILD_PYRAMIDS",
        calculate_statistics="CALCULATE_STATISTICS",
        build_thumbnails="NO_THUMBNAILS",
        operation_description="",
        force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
        estimate_statistics="ESTIMATE_STATISTICS",
        aux_inputs=None,
        enable_pixel_cache="NO_PIXEL_CACHE",
        cache_location=r"C:\Users\col12422\AppData\Local\Temp\13\ArcGISProTemp15064\Kangaroo_Island_2019"
    )
</details>

#### Mask Raster to ROI

1. Open the 'Extract by Mask' Geoprocessing Tool.
2. Input the following parameters:
    - **Input Raster** - Kangaroo_Island_2019
    - **Input Raster or Feature Mask Data** - Kangaroo_Island
    - **Output Raster** - Kangaroo_Island_2019_Masked
    - **Extraction Area** - Inside
3. WIthout running the tool, select the 'Copy Python Command' button and paste the results below
*Note: Your file paths will be different than the solution*

In [17]:
# Enter Solution

<details>
<summary> <b> Reveal Answer </b> </summary>

    with arcpy.EnvManager(scratchWorkspace=r"C:\Users\col12422\Documents\ArcGIS\Projects\Arcpy Workshop\Arcpy Workshop.gdb"):
        masked_raster = arcpy.sa.ExtractByMask(
            in_raster="Kangaroo_Island_2019",
            in_mask_data="Kangaroo_Island",
            extraction_area="INSIDE",
            analysis_extent='637977.776913391 6003103.86212958 784131.326170237 6063647.6465892 PROJCS["WGS_1984_UTM_Zone_53S",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",10000000.0],PARAMETER["Central_Meridian",135.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
        )
        masked_raster.save(r"C:\Users\col12422\Documents\ArcGIS\Projects\Arcpy Workshop\Arcpy Workshop.gdb\Kangaroo_Island_2019_Masked")
</details>

### Exercise 2: Thinking Ahead
In the previous exercise, we produced a repeatable workflow to mask Sentinel-2 imagery on Kangaroo Island. The 'Copy Python Command' tool is a great starting point for creating repeatable workflows on my own machine, but it has some flaws. Most notably, consider what would happen if trying to run this Notebook on another machine or with with different input rasters? Let's explore how we can make this code 'better' by using relative file paths and inputs. We'll recreate this workflow for Sentinel-2 imagery from 2020.

**arcpy.env.workspace** - Arcpy's method for identifying the project's workspace. The default workspace environment for every project is its default file geodatabase, in this case arcpy_workshop.gdb

In [99]:
arcpy.env.workspace

'C:\\Users\\col12422\\Documents\\ArcGIS\\Projects\\Arcpy Workshop\\Arcpy Workshop.gdb'

**os.path.join** - The os module is part of the Python standard library. os.path.join is a helpful way for joining filepaths to files in an os "safe" way.

In [100]:
import os
os.path.join(arcpy.env.workspace, "example.txt")

'C:\\Users\\col12422\\Documents\\ArcGIS\\Projects\\Arcpy Workshop\\Arcpy Workshop.gdb\\example.txt'

#### Global Variables

1. Use **arcpy.env.workspace** to create a workspace variable of the project's default location ```WORKSPACE = arcpy.env.workspace```
2. Create a mask variable of the Kangaroo Island boundary feature class ```MASK = 'Kangaroo_Island'```
3. Create a variable to name the mosaic dataset ```MOSAIC_DATASET_NAME = 'Kangaroo_Island_2020'```
4. Use **os.path.join** to create a mosaic dataset path variable from the project's workspace and the mosaic dataset name ```MOSAIC_DATASET_PATH = os.path.join(WORKSPACE, MOSAIC_DATASET_NAME)```
5. Use **os.path.join** to create two variables, one for raster input A, and the other for raster input B <br> ```INPUT_RASTER_A = os.path.join(WORKSPACE, MOSAIC_DATASET_NAME + 'A')``` <br> ```INPUT_RASTER_B = os.path.join(WORKSPACE, MOSAIC_DATASET_NAME + 'B')```

In [20]:
# Enter Solution

<details>
<summary> <b> Reveal Answer </b> </summary>

    WORKSPACE = arcpy.env.workspace

    MASK = 'Kangaroo_Island'

    MOSAIC_DATASET_NAME = 'Kangaroo_Island_2020'
    MOSAIC_DATASET_PATH = os.path.join(WORKSPACE, MOSAIC_DATASET_NAME)

    INPUT_RASTER_A = os.path.join(WORKSPACE, MOSAIC_DATASET_NAME + 'A')
    INPUT_RASTER_B = os.path.join(WORKSPACE, MOSAIC_DATASET_NAME + 'B') 
</details>

#### Create Mosaic Dataset

1. Input the variable for the workspace
2. Input the variable for the mosaic dataset name

In [ ]:
arcpy.management.CreateMosaicDataset(
    in_workspace= #1,
    in_mosaicdataset_name= #2,
    coordinate_system='PROJCS["WGS_1984_UTM_Zone_53S",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",10000000.0],PARAMETER["Central_Meridian",135.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    num_bands=None,
    pixel_type="",
    product_definition="NONE",
    product_band_definitions=None
)

<details>
<summary> <b> Reveal Answer </b> </summary>

    arcpy.management.CreateMosaicDataset(
        in_workspace=WORKSPACE,
        in_mosaicdataset_name=MOSAIC_DATASET_NAME,
        coordinate_system='PROJCS["WGS_1984_UTM_Zone_53S",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",10000000.0],PARAMETER["Central_Meridian",135.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
        num_bands=None,
        pixel_type="",
        product_definition="NONE",
        product_band_definitions=None
    )
</details>

#### Add Rasters to Mosaic Dataset

1. Input the variable for the mosaic dataset name
2. Input the variable for input raster A
3. Input the variable for input raster B

In [ ]:
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset= #1,
    raster_type="Raster Dataset",
    input_path=r"'{0}';'{1}'".format(#2, #3),
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="BUILD_PYRAMIDS",
    calculate_statistics="CALCULATE_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="ESTIMATE_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
)

<details>
<summary> <b> Reveal Answer </b> </summary>

    arcpy.management.AddRastersToMosaicDataset(
        in_mosaic_dataset=MOSAIC_DATASET_NAME,
        raster_type="Raster Dataset",
        input_path=r"'{0}';'{1}'".format(INPUT_RASTER_A, INPUT_RASTER_B),
        update_cellsize_ranges="UPDATE_CELL_SIZES",
        update_boundary="UPDATE_BOUNDARY",
        update_overviews="NO_OVERVIEWS",
        maximum_pyramid_levels=None,
        maximum_cell_size=0,
        minimum_dimension=1500,
        spatial_reference=None,
        filter="",
        sub_folder="SUBFOLDERS",
        duplicate_items_action="ALLOW_DUPLICATES",
        build_pyramids="BUILD_PYRAMIDS",
        calculate_statistics="CALCULATE_STATISTICS",
        build_thumbnails="NO_THUMBNAILS",
        operation_description="",
        force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
        estimate_statistics="ESTIMATE_STATISTICS",
        aux_inputs=None,
        enable_pixel_cache="NO_PIXEL_CACHE",
    )
</details>

#### Mask Raster to ROI

1. Input the variable for the workspace
2. Input the variable for the mosaic dataset name
3. Input the variable for the mask
4. Input the variable for the mosaic dataset name

In [ ]:
with arcpy.EnvManager(scratchWorkspace= #1):
    masked_raster = arcpy.sa.ExtractByMask(
        in_raster= #2,
        in_mask_data= #3,
        extraction_area="INSIDE",
        analysis_extent='637977.776913391 6003103.86212958 784131.326170237 6063647.6465892 PROJCS["WGS_1984_UTM_Zone_53S",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",10000000.0],PARAMETER["Central_Meridian",135.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
    )
    masked_raster.save( #4 + "_Masked")

<details>
<summary> <b> Reveal Answer </b> </summary>

    with arcpy.EnvManager(scratchWorkspace=WORKSPACE):
        masked_raster = arcpy.sa.ExtractByMask(
            in_raster=MOSAIC_DATASET_NAME,
            in_mask_data=MASK,
            extraction_area="INSIDE",
            analysis_extent='637977.776913391 6003103.86212958 784131.326170237 6063647.6465892 PROJCS["WGS_1984_UTM_Zone_53S",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",10000000.0],PARAMETER["Central_Meridian",135.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
        )
        masked_raster.save(MOSAIC_DATASET_NAME + "_Masked")
</details>